In [ ]:
from source_files_extended import *
from plot_helpers import *

from raster_compare.plots import PlotBase

import math

In [ ]:
from source_files import HILLSHADE_SNOW_ON
hillshade_snow_on = dem.hill_shade(**HILLSHADE_SNOW_ON)

# Snow Depth Difference

In [ ]:
HIST_BIN_WIDTH = .10
bins = np.concatenate((
#     [math.floor(np.nanmin(sfm_snow_depth_values))],
    np.arange(0, 4. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(np.nanmax(sfm_snow_depth_values))],
))

In [ ]:
AREA_PLOT_OPTS = dict(
    nrows=1, ncols=2, sharex=True, sharey=True, figsize=(17,14), dpi=1000
)

def area_plot():
    fig, axes = plt.subplots(**AREA_PLOT_OPTS)

    for ax in axes:
        ax.tick_params(axis='both', direction='inout', length=7)
        ax.ticklabel_format(axis='both', style='plain')
        ax.xaxis.set_ticks_position('both')
        ax.yaxis.set_ticks_position('both')
        
        ax.add_artist(ScaleBar(1.0, location='lower left', pad=0.5))
        ax.annotate('N', size=16, xy=(323800, 4319000), xytext=(323900, 4321000), ha="center", va="center", 
                    arrowprops=dict(arrowstyle="wedge,tail_width=1.5", facecolor='black'))
        
#         ax.set_facecolor('whitesmoke')
        ax.imshow(
            hillshade_snow_on,
            extent=sfm_snow_depth.extent,
            cmap='gray', clim=(1, 255), alpha=0.25,
        )
    return fig, axes

In [ ]:
COLORBAR_POSITION = dict(
    right=0.90, rect=[0.91, 0.262, 0.022, 0.52],
)

SFM_UNDER = dict(color='darkorange', alpha=0.4)
BLUE_CMAP.set_under(**SFM_UNDER)

imshow_opts = dict(
    extent=sfm_snow_depth.extent,
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=BLUE_CMAP.N,
    ),
    cmap=BLUE_CMAP,
)

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sfm_snow_depth_values,
    **imshow_opts
)
ax1.annotate('a)', xy=(335600, 4306300), fontsize=20)
ax1.add_artist(mpatches.Circle((323000, 4318500), 250, **SFM_UNDER))
ax1.annotate('No SfM snow depth', xy=(323400, 4318650), fontsize=14)

im_data = ax2.imshow(
    aso_snow_depth_values,
    **imshow_opts,
)
ax2.annotate('b)', xy=(335600, 4306300), fontsize=20)
cax = PlotBase.insert_colorbar(
    ax2, 
    im_data,
    SNOW_DEPTH_LABEL,
    ticks=np.arange(0, 5, 1),
    labelpad=20,
    **COLORBAR_POSITION
)

del im_data

In [ ]:
CASI_COLORS = ['dodgerblue', 'forestgreen', 'brown', 'orange']

In [ ]:
sd_difference_values = sfm_snow_depth_values - aso_snow_depth_values

np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    sd_difference_values,
    copy=False
)
classification_plot = np.ma.masked_where(
    sfm_snow_depth_values <= 0.0,
    casi_classification,
).astype(np.int8);

In [ ]:
bins = np.concatenate((
    [math.floor(np.nanmin(sd_difference_values))],
    np.arange(-2., 2. + HIST_BIN_WIDTH, HIST_BIN_WIDTH),
    [math.ceil(np.nanmax(sd_difference_values))],
))
imshow_opts = dict(
    extent=sfm_snow_depth.extent,
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=RED_BLUE_CMAP.N,
    ),
    cmap=RED_BLUE_CMAP,
)

In [ ]:
fig, (ax1, ax2) = area_plot()
fig.subplots_adjust(wspace=.15)

im_data = ax1.imshow(
    sd_difference_values,     
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")
PlotBase.insert_colorbar(ax1, im_data, 'Snow Depth Difference (m)')

im_data = ax2.imshow(
    classification_plot, 
    extent=sfm_snow_depth.extent,
    cmap=colors.ListedColormap(CASI_COLORS),
    alpha=0.8,
)
ax2.set_title("Snow Depth Differences - Classification")
PlotBase.insert_colorbar(ax2, im_data, 'Classification');

In [ ]:
high_elevation = np.ma.masked_where(
    dem_values <= 3500,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    dem_values > 3500,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()

ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - High Elevation >= 3500m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - Low Elevation < 3500")

PlotBase.insert_colorbar(ax2, im_data, 'Snow Depth Difference (m)', **COLORBAR_POSITION);

In [ ]:
high_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3100, 3500).mask,
    sd_difference_values,
)
low_elevation = np.ma.masked_where(
    np.ma.masked_outside(dem_values, 3800, 3900).mask,
    sd_difference_values,
)

fig, (ax1, ax2) = area_plot()
ax1.imshow(
    high_elevation, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences - 3100m < Elevation < 3200m")

im_data = ax2.imshow(
    low_elevation, 
    **imshow_opts,
)
ax2.set_title("Snow Depth Differences - 3800 < Elevation < 3900")

insert_sd_colorbar(ax2, im_data);

In [ ]:
fig, (ax1, ax2) = area_plot()

ax1.imshow(
    sd_difference_values, 
    **imshow_opts
)
ax1.set_title("Snow Depth Differences")

im_data = ax2.imshow(
    sfm_snow_free_values - dem_values, 
    **imshow_opts,
)
ax2.set_title("Snow Free - Reference DEM")

PlotBase.insert_colorbar(ax2, im_data, **COLORBAR_POSITION);

In [ ]:
data = [
    {
        'data': sfm_snow_free_values - dem_values,
        'label': 'SfM snow free - DEM',
        'color': 'dodgerblue',
    },
]

with plot_histogram(data, (-6, 6), figsize=(10, 8)) as ax:
    ax.set_title('Elevation Differences (SFM snow free - Reference DEM)');